<a href="https://colab.research.google.com/github/MohamedElsayed002/DeepLearning_IBM/blob/master/RNNDEMO19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Demo

## Using RNNs to classify sentiment on IMDB data

For this exercies we will train a "vanialla" RNN to predict the sentiment on IMDB reviews. our data consists of 25000 training sequences and 25000 test sequences the outcome is binary (postive/negative) and both outcomes are equally represented in both the trainnig and the test set.

Keras provides a convenient interafce to load the data immediately encode the words into integers (based on the most common words). this will save us alot of re drudgery that is usually involved when working with raw text

We will walk through the preparation of the data and the building an RNN model. Then it will be your turn to build your own models (and prepare the data how you see fit)

In [ ]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [ ]:
max_features = 20000 # This is used in the loading the data,
maxLen = 30 # maximum length of a sequence
batch_size = 32

In [ ]:
# Load in the data. the function automatically tokenizes the text into distinct integeres
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_features)
print(len(x_train),'train sequences')
print(len(x_test),'test sequences')

17464789/17464789 [==============================] - 1s 0us/step
25000 train sequences
25000 test sequences


In [ ]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxLen)
x_test = pad_sequences(x_test, maxlen=maxLen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [ ]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.





In [ ]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features,word_embedding_dim)) # This layer takes each integer in the sequence and embeds it in a 50-dimensional vector.
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1000286 (3.82 MB)
Trainable params: 1000286 (3.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
                  optimizer=rmsprop,
                  metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=10,
              validation_data=(x_test,y_test))

Epoch 1/10
782/782 [==============================] - 26s 31ms/step - loss: 0.6777 - accuracy: 0.5908 - val_loss: 0.6439 - val_accuracy: 0.6513
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 0.6033 - accuracy: 0.6812 - val_loss: 0.5775 - val_accuracy: 0.7002
Epoch 3/10
782/782 [==============================] - 16s 20ms/step - loss: 0.5380 - accuracy: 0.7309 - val_loss: 0.5388 - val_accuracy: 0.7267
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4931 - accuracy: 0.7622 - val_loss: 0.5147 - val_accuracy: 0.7438
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.4349 - accuracy: 0.8011 - val_loss: 0.4801 - val_accuracy: 0.7659
Epoch 7/10
782/782 [==============================] - 21s 27ms/step - loss: 0.4152 - accuracy: 0.8134 - val_loss: 0.4725 - val_accuracy: 0.7709
Epoch 8/10
782/782 [==============================] - 17s 22ms/step - loss: 0.3996 - accuracy: 0.8182 - val_loss: 0.4651 - val_accuracy:

In [ ]:
score , acc = model_rnn.evaluate(x_test,y_test,
                                 batch_size=batch_size)

print('Test score:', score)
print('Test accuracy,', acc)

782/782 [==============================] - 3s 3ms/step - loss: 0.4555 - accuracy: 0.7809
Test score: 0.4554561674594879
Test accuracy, 0.7809200286865234


# Exercies

In this ecerices, we will illustrate:

- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [ ]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 30s 37ms/step - loss: 0.6713 - accuracy: 0.5907 - val_loss: 0.6156 - val_accuracy: 0.6819
Epoch 2/10
782/782 [==============================] - 27s 34ms/step - loss: 0.5423 - accuracy: 0.7329 - val_loss: 0.5205 - val_accuracy: 0.7466
Epoch 3/10
782/782 [==============================] - 25s 32ms/step - loss: 0.4541 - accuracy: 0.7897 - val_loss: 0.4749 - val_accuracy: 0.7693
Epoch 4/10
782/782 [==============================] - 27s 34ms/step - loss: 0.4025 - accuracy: 0.8219 - val_loss: 0.4385 - val_accuracy: 0.7950
Epoch 5/10
782/782 [==============================] - 26s 33ms/step - loss: 0.3668 - accuracy: 0.8408 - val_loss: 0.4188 - val_accuracy: 0.8065
Epoch 6/10
782/782 [==============================] - 31s 39ms/step - loss: 0.3386 - accuracy: 0.8547 - val_loss: 0.4060 - val_accuracy: 0.8125
Epoch 7/10
782/782 [==============================] - 25s 32ms/step - loss: 0.3163 - accuracy: 0.8669 - val_loss: 0.3965 - val_accuracy:

In [ ]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 23s 28ms/step - loss: 0.6548 - accuracy: 0.6018 - val_loss: 0.5956 - val_accuracy: 0.6803
Epoch 2/10
782/782 [==============================] - 21s 27ms/step - loss: 0.5219 - accuracy: 0.7419 - val_loss: 0.4903 - val_accuracy: 0.7693
Epoch 3/10
782/782 [==============================] - 19s 24ms/step - loss: 0.4420 - accuracy: 0.7991 - val_loss: 0.4479 - val_accuracy: 0.7841
Epoch 4/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3978 - accuracy: 0.8237 - val_loss: 0.4222 - val_accuracy: 0.8046
Epoch 5/10
782/782 [==============================] - 18s 24ms/step - loss: 0.3729 - accuracy: 0.8350 - val_loss: 0.4018 - val_accuracy: 0.8162
Epoch 6/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3549 - accuracy: 0.8449 - val_loss: 0.4080 - val_accuracy: 0.8117
Epoch 7/10
782/782 [==============================] - 18s 24ms/step - loss: 0.3410 - accuracy: 0.8532 - val_loss: 0.3880 - val_accuracy:

In [ ]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3065 - accuracy: 0.8708 - val_loss: 0.4021 - val_accuracy: 0.8189
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3009 - accuracy: 0.8736 - val_loss: 0.3841 - val_accuracy: 0.8280
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2956 - accuracy: 0.8768 - val_loss: 0.3936 - val_accuracy: 0.8249
Epoch 4/10
782/782 [==============================] - 19s 25ms/step - loss: 0.2905 - accuracy: 0.8776 - val_loss: 0.3703 - val_accuracy: 0.8373
Epoch 5/10
782/782 [==============================] - 18s 24ms/step - loss: 0.2870 - accuracy: 0.8804 - val_loss: 0.3618 - val_accuracy: 0.8408
Epoch 6/10
782/782 [==============================] - 22s 28ms/step - loss: 0.2830 - accuracy: 0.8829 - val_loss: 0.3649 - val_accuracy: 0.8401
Epoch 7/10
782/782 [==============================] - 19s 24ms/step - loss: 0.2790 - accuracy: 0.8842 - val_loss: 0.3669 - val_accuracy: